In [1]:
import tensorflow as tf

import keras as k
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report

from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding,InputLayer
 

In [3]:
def process_sentence(sent):
    s = sent.strip().split()
    sp = None
    for i,word in enumerate(s):
        if word == 'EOS':
            sp = i
            break
    sent_tokens = s[1:sp]
    label = s[-1]
    return sent_tokens, label

In [5]:
from nltk import FreqDist

In [7]:
with open('ATIS_Train.txt','r')as fd:
    data_tr = fd.readlines()

In [9]:
with open('ATIS_Test.txt','r')as fd:
    data_te = fd.readlines()

In [71]:
x_tr_tokens = []
y_tr = []
x_te_tokens = []
y_te = []

for sent in data_tr:
    s, l = process_sentence(sent)
    x_tr_tokens.append(s)
    y_tr.append(l)
for sent in data_te:
    s, l = process_sentence(sent)
    x_te_tokens.append(s)
    y_te.append(l)

In [75]:
len(x_te_tokens)

893

In [18]:
cnt = 0
for l in y_tr:
    if '#' in l:
        cnt+=1

In [94]:
y_tr = [j.split('#')[0] for j in y_tr ]

In [95]:
y_te = [j.split('#')[0] for j in y_te ]

In [31]:
all_tokens = sum(x_tr_tokens,[])

In [36]:
fdist = FreqDist(all_tokens)

In [34]:
x_tr_tokens.__len__()

4978

In [40]:
fdist.most_common(4000)[-1]

('scenario', 1)

In [49]:
vocab = set(all_tokens)

In [47]:
word2idx = dict()

In [96]:
lab2ids = {lab:i for i,lab in enumerate(set(y_tr))}

In [97]:
lab2ids

{'atis_airline': 0,
 'atis_airport': 1,
 'atis_distance': 2,
 'atis_ground_service': 3,
 'atis_cheapest': 4,
 'atis_aircraft': 5,
 'atis_flight_time': 6,
 'atis_ground_fare': 7,
 'atis_city': 8,
 'atis_abbreviation': 9,
 'atis_airfare': 10,
 'atis_flight_no': 11,
 'atis_meal': 12,
 'atis_quantity': 13,
 'atis_flight': 14,
 'atis_capacity': 15,
 'atis_restriction': 16}

In [48]:
word2idx['<pad>'] = 0
word2idx['<unk>'] = 1

In [52]:
for i,word in enumerate(vocab):
    word2idx[word]=i+2

In [70]:
mxlen=0
for sent in x_tr_tokens:
    mxlen = max(len(sent),mxlen)

In [66]:
x_tr_ids = [ [word2idx[word] for word in sent] for sent in x_tr_tokens ]

In [76]:
x_te_ids = [ [word2idx.get(word,1) for word in sent] for sent in x_te_tokens ]

In [100]:
y_tr_idx = np.asarray([lab2ids[lab] for lab in y_tr ])
y_te_idx = np.asarray([lab2ids.get(lab,0) for lab in y_te ])

In [85]:
x_tr_ids_pad = []
for sent in x_tr_ids:
    ln = len(sent)
    toadd = mxlen-ln
    new = [0]*toadd
    new = sent+new
    x_tr_ids_pad.append(np.asarray(new,dtype = np.int32))

In [86]:
x_te_ids_pad = []
for sent in x_te_ids:
    sent = sent[:mxlen]
    ln = len(sent)
    toadd = mxlen-ln
    new = [0]*toadd
    new = sent+new
    x_te_ids_pad.append(np.asarray(new,dtype = np.int32))

In [87]:
x_te_ids_pad = np.asarray(x_te_ids_pad)
x_tr_ids_pad = np.asarray(x_tr_ids_pad)

In [88]:
x_te_ids_pad.shape

(893, 46)

In [89]:
x_tr_ids_pad.shape

(4978, 46)

In [104]:
y_tr_cat = to_categorical(y_tr_idx, num_classes=17)
y_te_cat = to_categorical(y_te_idx, num_classes=17)

In [111]:
act = [('relu','relu'),('','relu'),('relu','relu'),('relu','relu')]
print('binary with sigmoid')
model = Sequential()
model.add(InputLayer(input_shape=(mxlen,)))
model.add(Embedding(len(word2idx),100))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=17, activation='sigmoid'))
 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(x_tr_ids_pad, y_tr_cat, 
          epochs=3, batch_size=512, verbose=1,
          validation_split=0.2)
 


scores = model.evaluate(x_te_ids_pad, y_te_cat, verbose=1)
print("Accuracy:", scores[1]) 


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 46, 100)           90000     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 42, 64)            32064     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 64)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_2 (Dense)              (None, 17)                1105      
Total params: 156,001
Trainable params: 156,001
Non-trainable params: 0
________________________________________________

In [118]:
act = [('relu','relu'),('','relu'),('relu','relu'),('relu','relu')]
print('categorical with softmax')
model = Sequential()
model.add(InputLayer(input_shape=(mxlen,)))
model.add(Embedding(len(word2idx),100))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=17, activation='softmax'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(x_tr_ids_pad, y_tr_cat, 
          epochs=30, batch_size=512, verbose=1,
          validation_split=0.2)
 


scores = model.evaluate(x_te_ids_pad, y_te_cat, verbose=1)
y_hat = model.predict(x_te_ids_pad)

print("Accuracy:", scores[1])

categorical with softmax
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 46, 100)           90000     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 42, 64)            32064     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 8, 64)             0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_6 (Dense)              (None, 17)                1105      
Total params: 156,001
Trainable params: 156,001
Non-trainable params: 0
_______________________

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3982 samples, validate on 996 samples
Epoch 1/30
3982/3982 [==============================] - 0s 83us/step - loss: 2.5772 - accuracy: 0.6291 - val_loss: 2.1201 - val_accuracy: 0.7289
Epoch 2/30
3982/3982 [==============================] - 0s 41us/step - loss: 1.5893 - accuracy: 0.7436 - val_loss: 1.2270 - val_accuracy: 0.7289
Epoch 3/30
3982/3982 [==============================] - 0s 36us/step - loss: 1.2448 - accuracy: 0.7436 - val_loss: 1.1875 - val_accuracy: 0.7289
Epoch 4/30
3982/3982 [==============================] - 0s 34us/step - loss: 1.0971 - accuracy: 0.7436 - val_loss: 1.0926 - val_accuracy: 0.7289
Epoch 5/30
3982/3982 [==============================] - 0s 33us/step - loss: 1.0326 - accuracy: 0.7436 - val_loss: 1.0188 - val_accuracy: 0.7289
Epoch 6/30
3982/3982 [==============================] - 0s 34us/step - loss: 0.9621 - accuracy: 0.7436 - val_loss: 0.9716 - val_accuracy: 0.7289
Epoch 7/30
3982/3982 [==============================] - 0s 36us/step - loss: 0.8919

In [140]:
def get_report(y_true,y_hat,target_names):
    y_h = np.argmax(y_hat,axis=1)
    y_t = np.argmax(y_true,axis=1)
    print(classification_report(y_t, y_h, target_names=target_names[:16],labels = list(set(y_t))))

In [117]:
! pip install scipy

In [120]:
np.argmax(y_hat,axis=1).shape

(893,)

In [122]:
idx2lab = {j:i for i,j in lab2ids.items()}

In [127]:
tar_names = list(idx2lab.values())

In [129]:
y_hat.shape

(893, 17)

In [130]:
y_te_cat.shape

(893, 17)

In [141]:
get_report(y_te_cat,y_hat,tar_names)

                     precision    recall  f1-score   support

       atis_airline       0.97      0.90      0.94        40
       atis_airport       1.00      0.39      0.56        18
      atis_distance       0.78      0.70      0.74        10
atis_ground_service       0.86      1.00      0.92        36
      atis_cheapest       0.75      0.67      0.71         9
      atis_aircraft       0.14      1.00      0.25         1
   atis_flight_time       0.33      0.57      0.42         7
   atis_ground_fare       0.00      0.00      0.00         6
          atis_city       0.80      1.00      0.89        33
  atis_abbreviation       0.90      0.88      0.89        49
       atis_airfare       0.00      0.00      0.00         9
     atis_flight_no       0.00      0.00      0.00         6
          atis_meal       0.29      0.67      0.40         3
      atis_quantity       0.96      0.99      0.97       645
        atis_flight       1.00      0.38      0.55        21

           accuracy   

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1870: UserWarning: labels size, 15, does not match size of target_names, 16
  .format(len(labels), len(target_names))
/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [147]:
y_t = np.argmax(y_te_cat,axis=1)

In [148]:
len(set(y_t))

15